In [1]:
def Distance(Vec_1, Vec_2):
    x1, y1, z1 = Vec_1
    x2, y2, z2 = Vec_2
    
    xcont = (x2 - x1)**2
    ycont = (y2 - y1)**2
    zcont = (z2 - z1)**2
    distance = np.sqrt(xcont + ycont + zcont)
    return distance

In [2]:
def TermQ(r_i, r_j, residue_i, residue_j):
    numerator = (r_i - r_j)**2/100
    Sigma_ij = 0.1*(abs(residue_i-residue_j)**0.15)
    if Sigma_ij == 0:
        return 0
    termQ = np.exp(-numerator/(2*Sigma_ij**2))
    return termQ

In [3]:
def CrossQ(coords_a, coords_b, name_ids, chain_ids, residue_ids):
    Contact_Cutoff = 3
    count_eligible_contact = 0
    TotalQ = 0
    for atom_i in range(len(coords_a)):
        for atom_j in range(len(coords_b)):
            if(abs(residue_ids[atom_i] - residue_ids[atom_j]) >= Contact_Cutoff and chain_ids[atom_i] == chain_ids[atom_j]):
                count_eligible_contact += 1
                Distance_a = Distance(coords_a[atom_i], coords_a[atom_j])
                Distance_b = Distance(coords_b[atom_i], coords_b[atom_j])
                TotalQ = TotalQ + TermQ(Distance_a, Distance_b, residue_ids[atom_i], residue_ids[atom_j])
    print(count_eligible_contact)
    return TotalQ/count_eligible_contact

In [4]:
def InterChainQ(coords_a, coords_b, name_ids, chain_ids, residue_ids):
    Contact_Cutoff = 3
    count_eligible_contact = 0
    TotalQ = 0
    for atom_i in range(len(coords_a)):
        for atom_j in range(len(coords_b)):
            if(chain_ids[atom_i] != chain_ids[atom_j]):
                count_eligible_contact += 1
                Distance_a = Distance(coords_a[atom_i], coords_a[atom_j])
                Distance_b = Distance(coords_b[atom_i], coords_b[atom_j])
                TotalQ = TotalQ + TermQ(Distance_a, Distance_b, residue_ids[atom_i], residue_ids[atom_j])
    print(count_eligible_contact)
    return TotalQ/count_eligible_contact

In [5]:
def printCoords(pathtoPDB1, pathtoPDB2, output_file = "coordsQValue.csv"):
    structure_1 = pr.parsePDB(pathtoPDB1)

    # Extract coordinates, residue names, and atom names for the first structure
    coords_1 = structure_1.getCoords()  # numpy array of coordinates
    residues_1 = structure_1.getResnums()  # residue names
    atom_names_1 = structure_1.getNames()  # atom names
    chains_1 = structure_1.getChids()
    
    # Repeat for the second PDB file
    structure_2 = pr.parsePDB(pathtoPDB2)

    coords_2 = structure_2.getCoords()
    residues_2 = structure_2.getResnums()
    atom_names_2 = structure_2.getNames()
    chains_2 = structure_2.getChids()

    # Define a set of DNA residue names    
    # Initialize new lists for the filtered data
    CA_coords_1 = []
    CA_residues_1 = []
    CA_atom_names_1 = []
    CA_chains_1 = []
    CA_indicies_1 = []

    CA_coords_2 = []
    CA_residues_2 = []
    CA_atom_names_2 = []
    CA_chains_2 = []
    CA_indicies_2 = []

    #1 based indexing to align with fragment memory notations
    CA_index_1 = 1
    CA_index_2 = 1
    
    # Loop through the indices and filter based on the condition
    for i in range(len(atom_names_1)):
        if atom_names_1[i] == "CA":
            CA_coords_1.append(coords_1[i])
            CA_residues_1.append(residues_1[i])
            CA_atom_names_1.append(atom_names_1[i])
            CA_chains_1.append(chains_1[i])
            CA_indicies_1.append(CA_index_1)
            CA_index_1 += 1
    for i in range(len(atom_names_2)):
        if atom_names_2[i] == "CA":
            CA_coords_2.append(coords_2[i])
            CA_residues_2.append(residues_2[i])
            CA_atom_names_2.append(atom_names_2[i])
            CA_chains_2.append(chains_2[i])
            CA_indicies_2.append(CA_index_2)
            CA_index_2 += 1

    if (CA_index_1 != CA_index_2):
        print("Warning! Both files do not have the same number of atoms for Q values! Q Values may not be accurate!")

    with open(output_file, mode="w") as o:
        o.write(f"CA_index_1, x1, y1, z1, residues1, chain1, CA_index_2, x2, y2, z2, residues2, chain2 \n")
        for i in range(len(CA_coords_1)):
            o.write(f"""{CA_indicies_1[i]}, {CA_coords_1[i][0]}, {CA_coords_1[i][1]}, {CA_coords_1[i][2]}, {CA_residues_1[i]}, {CA_chains_1[i]}, {CA_indicies_2[i]}, {CA_coords_2[i][0]}, {CA_coords_2[i][1]}, {CA_coords_2[i][2]}, {CA_residues_2[i]}, {CA_chains_2[i]} \n""")

In [6]:
def calculateQ(pathtoPDB1, pathtoPDB2, printCoords=False, output_file = "coordsQValue.csv"):
    structure_1 = pr.parsePDB(pathtoPDB1)

    # Extract coordinates, residue names, and atom names for the first structure
    coords_1 = structure_1.getCoords()  # numpy array of coordinates
    residues_1 = structure_1.getResnums()  # residue names
    atom_names_1 = structure_1.getNames()  # atom names
    chains_1 = structure_1.getChids()
    
    # Repeat for the second PDB file
    structure_2 = pr.parsePDB(pathtoPDB2)

    coords_2 = structure_2.getCoords()
    residues_2 = structure_2.getResnums()
    atom_names_2 = structure_2.getNames()
    chains_2 = structure_2.getChids()

    # Define a set of DNA residue names    
    # Initialize new lists for the filtered data
    CA_coords_1 = []
    CA_residues_1 = []
    CA_atom_names_1 = []
    CA_chains_1 = []
    CA_indicies_1 = []

    CA_coords_2 = []
    CA_residues_2 = []
    CA_atom_names_2 = []
    CA_chains_2 = []
    CA_indicies_2 = []

    #1 based indexing to align with fragment memory notations
    CA_index_1 = 1
    CA_index_2 = 1
    
    # Loop through the indices and filter based on the condition
    for i in range(len(atom_names_1)):
        if atom_names_1[i] == "CA":
            CA_coords_1.append(coords_1[i])
            CA_residues_1.append(residues_1[i])
            CA_atom_names_1.append(atom_names_1[i])
            CA_chains_1.append(chains_1[i])
            CA_indicies_1.append(CA_index_1)
            CA_index_1 += 1
    for i in range(len(atom_names_2)):
        if atom_names_2[i] == "CA":
            CA_coords_2.append(coords_2[i])
            CA_residues_2.append(residues_2[i])
            CA_atom_names_2.append(atom_names_2[i])
            CA_chains_2.append(chains_2[i])
            CA_indicies_2.append(CA_index_2)
            CA_index_2 += 1

    if (CA_index_1 != CA_index_2):
        print("Warning! Both files do not have the same number of atoms for Q values! Q Values may not be accurate")

    if printCoords:
        with open(output_file, mode="w") as o:
            o.write(f"CA_index_1, x1, y1, z1, residues1, chain1, CA_index_2, x2, y2, z2, residues2, chain2 \n")
            for i in range(len(CA_coords_1)):
                o.write(f"""{CA_indicies_1[i]}, {CA_coords_1[i][0]}, {CA_coords_1[i][1]}, {CA_coords_1[i][2]}, {CA_residues_1[i]}, {CA_chains_1[i]}, {CA_indicies_2[i]}, {CA_coords_2[i][0]}, {CA_coords_2[i][1]}, {CA_coords_2[i][2]}, {CA_residues_2[i]}, {CA_chains_2[i]} \n""")
    
    return CrossQ(CA_coords_1, CA_coords_2, CA_atom_names_1, CA_chains_1, CA_residues_1)

In [ ]:
def calculateInterChainQ(pathtoPDB1, pathtoPDB2, printCoords=False, output_file = "coordsQValue_interChain.csv"):
    structure_1 = pr.parsePDB(pathtoPDB1)

    # Extract coordinates, residue names, and atom names for the first structure
    coords_1 = structure_1.getCoords()  # numpy array of coordinates
    residues_1 = structure_1.getResnums()  # residue names
    atom_names_1 = structure_1.getNames()  # atom names
    chains_1 = structure_1.getChids()
    
    # Repeat for the second PDB file
    structure_2 = pr.parsePDB(pathtoPDB2)

    coords_2 = structure_2.getCoords()
    residues_2 = structure_2.getResnums()
    atom_names_2 = structure_2.getNames()
    chains_2 = structure_2.getChids()

    # Define a set of DNA residue names    
    # Initialize new lists for the filtered data
    CA_coords_1 = []
    CA_residues_1 = []
    CA_atom_names_1 = []
    CA_chains_1 = []
    CA_indicies_1 = []

    CA_coords_2 = []
    CA_residues_2 = []
    CA_atom_names_2 = []
    CA_chains_2 = []
    CA_indicies_2 = []

    #1 based indexing to align with fragment memory notations
    CA_index_1 = 1
    CA_index_2 = 1
    
    # Loop through the indices and filter based on the condition
    for i in range(len(atom_names_1)):
        if atom_names_1[i] == "CA":
            CA_coords_1.append(coords_1[i])
            CA_residues_1.append(residues_1[i])
            CA_atom_names_1.append(atom_names_1[i])
            CA_chains_1.append(chains_1[i])
            CA_indicies_1.append(CA_index_1)
            CA_index_1 += 1
    for i in range(len(atom_names_2)):
        if atom_names_2[i] == "CA":
            CA_coords_2.append(coords_2[i])
            CA_residues_2.append(residues_2[i])
            CA_atom_names_2.append(atom_names_2[i])
            CA_chains_2.append(chains_2[i])
            CA_indicies_2.append(CA_index_2)
            CA_index_2 += 1

    if (CA_index_1 != CA_index_2):
        print("Warning! Both files do not have the same number of atoms for Q values! Q Values may not be accurate")

    if printCoords:
        with open(output_file, mode="w") as o:
            o.write(f"CA_index_1, x1, y1, z1, residues1, chain1, CA_index_2, x2, y2, z2, residues2, chain2 \n")
            for i in range(len(CA_coords_1)):
                o.write(f"""{CA_indicies_1[i]}, {CA_coords_1[i][0]}, {CA_coords_1[i][1]}, {CA_coords_1[i][2]}, {CA_residues_1[i]}, {CA_chains_1[i]}, {CA_indicies_2[i]}, {CA_coords_2[i][0]}, {CA_coords_2[i][1]}, {CA_coords_2[i][2]}, {CA_residues_2[i]}, {CA_chains_2[i]} \n""")
    
    return InterChainQ(CA_coords_1, CA_coords_2, CA_atom_names_1, CA_chains_1, CA_residues_1)

In [10]:
import numpy as np
import prody as pr
import pandas as pd

AWSEM_folder = "../../frags_lib/DNA_NFkB_FL_19_291"
reference = f"{AWSEM_folder}/crystal_structure-openmmawsem.pdb"

test_sims = ["DNA_NFkB_FL_19_291-bs1", "DNA_NFkB_FL_19_291-bs2", "DNA_NFkB_FL_19_291-bs3-bs1coord", "DNA_NFkB_FL_19_291-bs3-bs2coord"]

for test_sim in test_sims:
    for i in range(1,9):
        data = pd.read_csv(f'{test_sim}/run_{i}/info.dat', sep='\s+')
        control = data['Q'].iloc[-1]
        print(f'r{i}', f"""control = {control}', 
        f"thisScript = {calculateInterChainQ(reference, f'{test_sim}/run_{i}/Last_Frame.pdb', printCoords=True, output_file=f'qvaloutput/{test_sim}_r{i}_InterChain.txt')}""")

<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_468981/2801070324.py:12: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_csv(f'{test_sim}/run_{i}/info.dat', sep='\s+')


171524
r1 control = 0.3', 
        f"thisScript = 0.08941197628728265
171524
r2 control = 0.3', 
        f"thisScript = 0.13144660036175543
171524
r3 control = 0.34', 
        f"thisScript = 0.12748166529629545
171524
r4 control = 0.33', 
        f"thisScript = 0.11459403820676728
171524
r5 control = 0.27', 
        f"thisScript = 0.07982706794715659
171524
r6 control = 0.28', 
        f"thisScript = 0.07499562406087194
171524
r7 control = 0.31', 
        f"thisScript = 0.13280792123630275
171524
r8 control = 0.34', 
        f"thisScript = 0.1279648788593501
171524
r1 control = 0.32', 
        f"thisScript = 0.10858510741741242
171524
r2 control = 0.3', 
        f"thisScript = 0.07121807568299522
171524
r3 control = 0.28', 
        f"thisScript = 0.1007662527951125
171524
r4 control = 0.3', 
        f"thisScript = 0.10201607794036363
171524
r5 control = 0.32', 
        f"thisScript = 0.12306050546152007
171524
r6 control = 0.28', 
        f"thisScript = 0.08665263131232191
171524
r7 co

In [11]:
import numpy as np
import prody as pr
import pandas as pd

pr.confProDy(verbosity='none')

AWSEM_folder = "../../frags_lib/DNA_NFkB_FL_19_291"
reference = f"{AWSEM_folder}/crystal_structure-openmmawsem.pdb"

test_sims = ["DNA_NFkB_FL_19_291-bs1", "DNA_NFkB_FL_19_291-bs2", "DNA_NFkB_FL_19_291-bs3-bs1coord", "DNA_NFkB_FL_19_291-bs3-bs2coord"]

for test_sim in test_sims:
    for i in range(1,9):
        data = pd.read_csv(f'{test_sim}/run_{i}/info.dat', sep='\s+')
        control = data['Q'].iloc[-1]
        print(f'r{i}', f'control = {control}', f"thisScript = {calculateQ(reference, f'{test_sim}/run_{i}/Last_Frame.pdb')}")

<>:14: SyntaxWarning: invalid escape sequence '\s'
<>:14: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_468981/1281087296.py:14: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_csv(f'{test_sim}/run_{i}/info.dat', sep='\s+')


170122
r1 control = 0.3 thisScript = 0.30477142269757745
170122
r2 control = 0.3 thisScript = 0.30279347263110595
170122
r3 control = 0.34 thisScript = 0.3390139506471637
170122
r4 control = 0.33 thisScript = 0.3264292234406385
170122
r5 control = 0.27 thisScript = 0.27339056547478974
170122
r6 control = 0.28 thisScript = 0.2750897182862293
170122
r7 control = 0.31 thisScript = 0.3148318916690169
170122
r8 control = 0.34 thisScript = 0.34265967376181994
170122
r1 control = 0.32 thisScript = 0.3208554247525533
170122
r2 control = 0.3 thisScript = 0.2976813702732331
170122
r3 control = 0.28 thisScript = 0.283498451285487
170122
r4 control = 0.3 thisScript = 0.30179347856569955
170122
r5 control = 0.32 thisScript = 0.3172529864954771
170122
r6 control = 0.28 thisScript = 0.2795474309160446
170122
r7 control = 0.29 thisScript = 0.2878940479094367
170122
r8 control = 0.28 thisScript = 0.2795700742649551
170122
r1 control = 0.3 thisScript = 0.29831978579996987
170122
r2 control = 0.3 thisScr